In [2]:
import eikon as ek
import pandas as pd
ek.set_app_key("5f62315c837645a78ff033f35644b3ba3ef5753a")

In [3]:
import time
def fetch_data_in_batches(tickers, batch_size, file_output_name, year=None):
    if year is None:
        year = "FY0"
    else:
        year = "FY" + str(year)

    def get_data_batch(batch):
        try:
            data, _ = ek.get_data(
                batch,
                fields=[
                    f'TR.BGS.BUSTotalRevenue(SDate={year}).segmentName',f'TR.BGS.BUSTotalRevenue(SDate={year}).segmentCode',f"TR.Revenue(SDate={year},Frq=FY,Curn=USD)"
                ]
            )
            #add a column with the year
            data["year"] = year
            return data  # Return only the DataFrame
        except Exception as e:
            print(f"Failed to fetch data for batch: {batch}. Error: {e}")
            return None  # Return None in case of failure
        
    all_data_frames = []
    batches_to_process = [tickers[i:i + batch_size] for i in range(0, len(tickers), batch_size)]
    failed = []

    while batches_to_process:
        print("Number of batches to process:", len(batches_to_process)+1)
        new_failed = []
        for batch in batches_to_process:
            print("Getting data for batch number:", batches_to_process.index(batch)+1, "of", len(batches_to_process)+1)
            time.sleep(0.2)
            print("Processing batch")
            batch_data = get_data_batch(batch)
            
            if batch_data is not None:
                print("batch successful")
                all_data_frames.append(batch_data)
            else:
                print("batch failed")
                new_failed.append(batch)

        # Update the list of batches to process
        batches_to_process = new_failed

        # Remember the failed batches
        failed.extend(new_failed)

    # Concatenate all the individual batch data frames into one
    final_data_frame = pd.concat(all_data_frames, ignore_index=True) if all_data_frames else pd.DataFrame()

    # Save the data to a file (adjust the path and file name as needed)
    final_data_frame.to_csv(file_output_name, index=False)
    
    return final_data_frame, failed

In [4]:
data = pd.read_csv("event_companies.csv")

In [5]:
import pandas as pd

# Assuming df is your DataFrame
# Load your DataFrame here if it's not already loaded
# df = pd.read_csv('your_file.csv')

# Modified function with additional parameters
def check_segments(group, segment_codes, strings_to_check, min_revenue):
    # Check if any segment code is in this group
    has_segment_code = group['Segment Code'].str.contains('|'.join(segment_codes)).any()

    # Check if Revenue is over a certain amount
    has_revenue = group['Revenue'].gt(min_revenue).any()

    # Check if any segment name contains any of the strings
    has_string = any(string.lower() in group['Segment Name'].to_string() for string in strings_to_check)
    
    return pd.Series({'Has Segment Code': has_segment_code, 'Has Revenue': has_revenue, 'Has String': has_string})


# Define your list of segment codes and strings
segment_codes = ['221111','221113','221114','221115','221116',"221117", '221118',]  # Example segment codes
strings_to_check = ['renewable'," ren "]  # Example strings

# Group by 'Company Name' and apply the checking function with parameters
#results = df.groupby('Instrument').apply(check_segments, segment_codes, strings_to_check)

#print(results)


In [6]:
data =fetch_data_in_batches(list(data['companies']), 500, "segments.csv", 0)


Number of batches to process: 30
Getting data for batch number: 1 of 30
Processing batch


KeyboardInterrupt: 

In [ ]:
data[0]
segment_codes = ['221111','221114','221115','221116',"221117"]  # Example segment codes
strings_to_check = ['renewable'," ren "] 
min_revenue = 20000000
results = data[0].groupby('Instrument').apply(check_segments, segment_codes, strings_to_check,min_revenue)


In [ ]:
companies = results[((results['Has Segment Code'] == True)& (results["Has Revenue"]== True)) | ((results['Has String'] == True)&(results["Has Revenue"]== True))]


In [ ]:
#get the companies that have renewable segments
companies = companies.reset_index()
companies_list = companies[["Instrument"]]
list_companies = companies_list
list_companies.to_csv("companies_with_ren_segments_and_data.csv", index=False)

In [ ]:
list_companies

,Instrument
0,000537.SZ
1,000539.SZ
2,000629.SZ
3,000883.SZ
4,002060.SZ
...,...
120,TTPW.NS
121,UN01.DE
122,UNACEMC1.LM
123,VERB.VI


: 

In [ ]:
data

NameError: name 'data' is not defined